In [1]:
import torch
from scipy.stats import multivariate_normal
import numpy as np
from tqdm import tqdm

def normalize_2nd_moment(x, dim=1, eps=1e-8):
    x = torch.from_numpy(x)
    yo = x * (x.square().mean(dim=dim, keepdim=True) + eps).rsqrt()
    yo = yo.numpy()
    return yo



In [2]:
d = 1024
mu = np.zeros(d)  # mean
covariance_matrix = np.eye(d)  # covariance matrix
z = np.random.normal(0,1,(100,d)) # sample
pdf_values = multivariate_normal.logpdf(z, mean=mu, cov=covariance_matrix)
print(pdf_values.mean())


-1458.4179903784172


In [3]:
covariance_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [4]:
a = torch.load("./new-vanilla-results/ffhq-og-no-snoise-10ff/train/latent/0-0.npy").numpy()
lolo = np.linalg.norm(a, axis=1)
print(lolo.mean())

31.720066


In [5]:
def pdf_test(directory):
    mean = []
    for i in tqdm(range(0,15000,1)):
        try:
            a = torch.load(f"./new-vanilla-results/{directory}/train/latent/0-{i}.npy").numpy()
            pdf_values = multivariate_normal.logpdf(a, mean=mu, cov=covariance_matrix)
            mean.append(pdf_values.mean())
        except:
            pass
    return np.array(mean).mean()

def pdf_test2(directory):
    mean = []
    for i in tqdm(range(0,15000,1)):
        try:
            a = torch.load(f"./new-vanilla-results/{directory}/train/latent/0-{i}.npy").numpy()
            pdf_values = np.power(np.linalg.norm(a, axis=1),2)
            mean.append(pdf_values.mean())
        except:
            pass
    return np.array(mean).mean()
    

In [6]:
print(pdf_test2("ffhq-og-no-snoise-10ff"))

  0%|          | 0/15000 [00:00<?, ?it/s]

100%|██████████| 15000/15000 [00:00<00:00, 38798.74it/s]

1024.0521


In [7]:
print(pdf_test2("ffhq-og-no-snoise-10ff-eps-18"))

100%|██████████| 15000/15000 [00:00<00:00, 40410.93it/s]

1023.8495


In [8]:
print(pdf_test2("ffhq-og-no-snoise-10ff-eps-15"))

100%|██████████| 15000/15000 [00:00<00:00, 15524.58it/s]

1023.92487


In [9]:
print(pdf_test2("ffhq-og-no-snoise-10ff-eps-12"))

100%|██████████| 15000/15000 [00:00<00:00, 36531.42it/s]

1023.9377


In [10]:
print(pdf_test2("ffhq-og-no-snoise-5ff"))

100%|██████████| 15000/15000 [00:00<00:00, 92291.09it/s]

1024.0272


In [11]:
print(pdf_test2("ffhq-og-no-snoise-20ff-2nd-normalized"))

100%|██████████| 15000/15000 [00:00<00:00, 47141.17it/s]

1023.9281
